In [1]:
from lerLaudo import lerLaudo
from LerSimuladorExcel import lerSimulador
from lerContrato import lerContrato, dadosParticipantes

In [2]:
contrato = r"C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Contratos\HE_Contrato_Agostinho_Assinatura Digital_VFinal.pdf"
laudo = r'C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Laudos\5. Laudo - Agostinho.pdf'
simulador = r'C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Simuladores\HE Simulador Agostinho.xlsm'
import warnings
#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')


In [3]:
dadosContrato = lerContrato(contrato)
dadosContrato

{'valorTotal': '185236.16',
 'tabela': 'SAC',
 'registro': '3042.67',
 'Taxa ao Mes': 0.1162,
 'valorLiquido': '169200.00',
 'prazoMes': '240',
 'valorPrimeiraParcelaComEncargos': '2754.90',
 'valorImóvel': '376000.00',
 'prazoContrato': '234',
 'ultimaParcela': '30/06/2042',
 'dataLiberação': '08/06/2022',
 'valorPrimeiraParcela': '2662.09',
 'primeiraParcela': '30/01/2023',
 'indice': 'IPCA',
 'Matrícula': '141.951',
 'Cartório': '02 de Registro Geral do 3º Oficial de Registro de Imóveis da Comarca de São Paulo/SP',
 'CCI': '0001013447/AFD',
 'Titular': 'AGOSTINHO FRANCISCO DE SOUZA ',
 'operação': 'PF',
 'Quantidade': 2,
 'Participante1': 'AGOSTINHO FRANCISCO DE SOUZA',
 'Participação1': '100%',
 'dataContrato': '06/06/2022'}

In [7]:
import os
files_dir = r"C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Contratos"
contratoFiles = [f for f in os.listdir(files_dir) if f.endswith(".pdf")]
#contratoFiles

In [8]:
# for contrato in contratoFiles:
#     contract_file = os.path.join(files_dir, contrato)
#     extracaoContrato = lerContrato(contract_file)
#     nome = extracaoContrato['Participante1']
#     infoParticipantes = dadosParticipantes(contract_file, extracaoContrato)
    #print(f'----------- {nome} -----------\n{extracaoContrato}\n\n{infoParticipantes}\n\n')
    

Código 1.1

In [4]:
import bdb
from errno import WSABASEERR
from tkinter import *
import tkinter as tk
from tkinter.font import BOLD
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
from time import sleep
import time
import os
from turtle import bgcolor, width 
import warnings

#Remover Avisos
warnings.filterwarnings('ignore', category=UserWarning, module='tkinter')

#Definir Variáveis Globais
contrato = None
simulador = None
laudo = None
textoContrato = None
botaoUm = None

ws = Tk()
ws.title('Validação Contrato x Simulador x Laudo')
ws.geometry('1920x1080') 

#Passo 1 - Criar um main frame
main_frame = Frame(ws)
main_frame.pack(fill=BOTH, expand=1)

#Passo 2 - Criar um canvas
my_canvas = Canvas(main_frame)
my_canvas.pack(side=LEFT, fill=BOTH, expand=1)

#Passo 3 - Adicionar a barra de rolagem ao canvas
scrollbar = Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
scrollbar.pack(side=RIGHT, fill=Y)

#Passo 4 - Configurar o canvas
my_canvas.configure(yscrollcommand=scrollbar.set)
my_canvas.bind('<Configure>', lambda e:my_canvas.configure(scrollregion=my_canvas.bbox("all")))

#Passo 5 - Criar um outro frame dentro do canvas
canvas = Frame(my_canvas)

#Passo 6 - Adicionar o novo frame dentro de uma janela no canvas
my_canvas.create_window((0,0), window=canvas, anchor='nw')



#Função Selecionar PDF
def open_file_pdf(entryBox):
    file_path = askopenfile(mode='r', filetypes=[('ALL', 'pdf')])
    entryBox.delete(0, 'end')
    if file_path is not None:
        pass
    entryBox.insert(END, file_path.name) # add this

#Função Selecionar Excel (xlsm)
def open_file_excel():
    textBoxSimulador.delete(0, 'end')
    file_path = askopenfile(mode='r', filetypes=[('ALL','*xlsm')])
    if file_path is not None:
        pass
    textBoxSimulador.insert(END, file_path.name) # add this

#Função para verificar se o path é válido
def verificarArquivo(arquivo):
    try:
        with open(arquivo, 'r') as f:
            print('Aquivo Existe')
    except IOError:
        print('Arquivo não encontrado!')

#Função para definir paths    
def savePaths():
    global contrato 
    global simulador
    global laudo

    contrato = textBoxContrato.get()
    laudo = textBoxLaudo.get()
    simulador = textBoxSimulador.get()
    if contrato != "" and laudo != "" and simulador != "":
        return 0
    else:
        return 1

#Ler o Contrato
def infosContrato(pathContrato):
    global textBoxContrato
    textoContrato = lerContrato(pathContrato)
    textoParticipantes = dadosParticipantes(pathContrato, textoContrato)
    
    #Separar Key-Value
    valorBrutoC = str(textoContrato['valorTotal']).replace(".",",")
    valorLiquidoC = str(textoContrato['valorLiquido']).replace(".",",")
    valorAmortizacaoC = str(textoContrato['tabela'])
    valorIndiceC = str(textoContrato['indice'])
    prazoC = str(textoContrato['prazoMes'])
    parcelasContratoC = str(textoContrato['prazoContrato'])
    dataDesembolcoC = str(textoContrato['dataLiberação'])
    dataPriParC = str(textoContrato['primeiraParcela'])
    dataUltimaParC = str(textoContrato['ultimaParcela'])
    valorPriParC = str(textoContrato['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosC = str(textoContrato['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoC = str(textoContrato['valorImóvel']).replace(".",",")
    matriculaImovelC = str(textoContrato['Matrícula'])
    cartorioImovelC = str(textoContrato['Cartório'])

    #Novas Extrações
    quantidadeC = str(textoContrato['Quantidade'])
    #dataContratoC = str(textoContrato['dataContrato'])

    #Clear Box
    tbCValorBruto.delete(0, 'end')
    tbCValorLiquido.delete(0, 'end')
    tbCAmortizacao.delete(0, 'end')
    tbCIndice.delete(0, 'end')
    tbCPrazo.delete(0, 'end')
    tbCParcelas.delete(0, 'end')
    tbCDataDesem.delete(0, 'end')
    tbCDataPriParcela.delete(0, 'end')
    tbCDataUltimaParcela.delete(0, 'end')
    tbCValorPriPar.delete(0, 'end')
    tbCValorPriEncargosPar.delete(0, 'end')
    tbCValorAvaliacao.delete(0, 'end')
    tbCMatricula.delete(0, 'end')
    tbCCartorio.delete(0, 'end')

    #Novos Casos
    tbCNumParticipantes.delete(0, 'end')
    #tbCDataContrato.delete(0, 'end')

    #Inserir dados do Contrato
    tbCValorBruto.insert(END, valorBrutoC)
    tbCValorLiquido.insert(END, valorLiquidoC)
    tbCAmortizacao.insert(END, valorAmortizacaoC)
    tbCIndice.insert(END, valorIndiceC)
    tbCPrazo.insert(END, prazoC)
    tbCParcelas.insert(END, parcelasContratoC)
    tbCDataDesem.insert(END, dataDesembolcoC)
    tbCDataPriParcela.insert(END, dataPriParC)
    tbCDataUltimaParcela.insert(END, dataUltimaParC)
    tbCValorPriPar.insert(END, valorPriParC)
    tbCValorPriEncargosPar.insert(END, valorPriParEncargosC)
    tbCValorAvaliacao.insert(END, valorAvaliacaoC)
    tbCMatricula.insert(END, matriculaImovelC)
    tbCCartorio.insert(END, cartorioImovelC)

    #Novos Casos
    #tbCDataContrato.insert(END, dataContratoC)
    tbCNumParticipantes.insert(END,quantidadeC)

    return textoContrato

#Ler Simulador
def infosSimulador(pathSimulador):
    textoSimulador = lerSimulador(pathSimulador)
    
    #Separar Key-Value
    valorBrutoS = str(round(textoSimulador['valorTotal'],2)).replace(".",",")
    valorLiquidoS = str(textoSimulador['valorLiquido']).replace(".",",")
    valorAmortizacaoS = str(textoSimulador['tabela'])
    valorIndiceS = str(textoSimulador['indice'])
    prazoS = str(textoSimulador['prazoMes'])
    parcelasContratoS = str(textoSimulador['prazoContrato'])
    dataDesembolcoS = str(textoSimulador['dataContrato'])
    dataPriParS = str(textoSimulador['primeiraParcela'])
    dataUltimaParS = str(textoSimulador['ultimaParcela'])
    valorPriParS = str(textoSimulador['valorPrimeiraParcela']).replace(".",",")
    valorPriParEncargosS = str(textoSimulador['valorPrimeiraParcelaComEncargos']).replace(".",",")
    valorAvaliacaoS = str(textoSimulador['valorImóvel']).replace(".",",")

    #Clear Box
    tbSValorBruto.delete(0, 'end')
    tbSValorLiquido.delete(0, 'end')
    tbSAmortizacao.delete(0, 'end')
    tbSIndice.delete(0, 'end')
    tbSPrazo.delete(0, 'end')
    tbSParcelas.delete(0, 'end')
    tbSDataDesem.delete(0, 'end')
    tbSDataPriParcela.delete(0, 'end')
    tbSDataUltimaParcela.delete(0, 'end')
    tbSValorPriPar.delete(0, 'end')
    tbSValorPriEncargosPar.delete(0, 'end')
    tbSValorAvaliacao.delete(0, 'end')

    #Inserir dados do Contrato
    tbSValorBruto.insert(END, valorBrutoS)
    tbSValorLiquido.insert(END, valorLiquidoS)
    tbSAmortizacao.insert(END, valorAmortizacaoS)
    tbSIndice.insert(END, valorIndiceS)
    tbSPrazo.insert(END, prazoS)
    tbSParcelas.insert(END, parcelasContratoS)
    tbSDataDesem.insert(END, dataDesembolcoS)
    tbSDataPriParcela.insert(END, dataPriParS)
    tbSDataUltimaParcela.insert(END, dataUltimaParS)
    tbSValorPriPar.insert(END, valorPriParS)
    tbSValorPriEncargosPar.insert(END, valorPriParEncargosS)
    tbSValorAvaliacao.insert(END, valorAvaliacaoS)

#Ler o Laudo
def infosLaudo(pathLaudo):
    textoLaudo = lerLaudo(pathLaudo)

    #Separar Key-Value
    valorAvaliacaoL = str(textoLaudo['Valor de Mercado: '])
    matriculaImovelL = str(textoLaudo['Matrícula: '])
    cartorioImovelL = str(textoLaudo['Cartório'])

    #Clear Box
    tbLValorAvaliacao.delete(0, 'end')
    tbLMatricula.delete(0, 'end')
    tbLCartorio.delete(0, 'end')

    #Inserir dados do Contrato
    tbLValorAvaliacao.insert(END, valorAvaliacaoL)
    tbLMatricula.insert(END, matriculaImovelL)
    tbLCartorio.insert(END, cartorioImovelL)

#Função Validar Itens
def validarEntry():
    
    #Var Globais Contrato
    global tbCValorBruto
    global tbCValorLiquido
    global tbCAmortizacao
    global tbCIndice
    global tbCPrazo
    global tbCParcelas
    global tbCDataDesem
    global tbCDataPriParcela
    global tbCDataUltimaParcela
    global tbCValorPriPar
    global tbCValorPriEncargosPar
    global tbCValorAvaliacao
    global tbCMatricula
    global tbCCartorio
    global tbCNumParticipantes
    global tbCDataContrato

    #Var Globais Simulador
    global tbSValorBruto
    global tbSValorLiquido
    global tbSAmortizacao
    global tbSIndice
    global tbSPrazo
    global tbSParcelas
    global tbSDataDesem
    global tbSDataPriParcela
    global tbSDataUltimaParcela
    global tbSValorPriPar
    global tbSValorPriEncargosPar
    global tbSValorAvaliacao

    #Var Globais Laudo
    global tbLValorAvaliacao
    global tbLMatricula
    global tbLCartorio

    #Definção de Lista de Contrato
    listaContrato = [tbCValorBruto, tbCValorLiquido, tbCAmortizacao, tbCIndice, tbCPrazo, tbCParcelas, tbCDataDesem, tbCDataPriParcela,
                    tbCDataUltimaParcela, tbCValorPriPar, tbCValorPriEncargosPar, tbCValorAvaliacao, tbCMatricula, tbCCartorio
                    ]

    #Definção de Lista do Simulador
    listaSimulador = [tbSValorBruto, tbSValorLiquido, tbSAmortizacao, tbSIndice, tbSPrazo, tbSParcelas, tbSDataDesem, tbSDataPriParcela,
                        tbSDataUltimaParcela, tbSValorPriPar, tbSValorPriEncargosPar, tbSValorAvaliacao, tbLMatricula, tbLCartorio
                    ]
    
    tamanhoLista = len(listaContrato)

    for i in range(tamanhoLista):
        num = i + 1
        varContrato = listaContrato[i].get() 
        varSimulador = listaSimulador[i].get()
        
        #Corrigir Espaço Indesejado
        if i<13:
            varContrato = varContrato.replace(" ", "")
            varSimulador = varSimulador.replace(" ", "")
        
        #Transformar Valores Númericos
        if i == 0 or i == 1 or i == 9 or i == 10 or i == 11:
            varContrato = round(float(varContrato.replace(",",".")),2)
            varSimulador = round(float(varSimulador.replace(",",".")),2)

        if i == 2 or i == 3:
            varContrato = varContrato.lower()
            varSimulador = varSimulador.lower()

        #Comparação de Valores
        if varContrato == varSimulador:
            botaoUm = "VERDADEIRO"
            color = 'green'
            fontColor = 'white'
            
        else:
            botaoUm = "FALSO"
            color = 'red'
            fontColor = 'white'
    
        # #Criar Botão de Validação
        validValorBruto = tk.Button(
             canvas,
             text = botaoUm,
             bg=color,
             fg=fontColor
             )
        if i<11:
            validValorBruto.grid(row=8+num, column=2)
        else:
            validValorBruto.grid(row=12+num, column=2)

#Função UploadFiles
def uploadFiles():
    global contrato
    pb1 = Progressbar(
        canvas, 
        orient=HORIZONTAL, 
        length=300, 
        mode='determinate'
        )
    pb1.grid(row=4, columnspan=3, pady=20)
    for i in range(5):
        canvas.update_idletasks()
        pb1['value'] += 20
        time.sleep(1)
    pb1.destroy()
    msg = savePaths()
    if msg == 0:
        textoUpload = "Todos os arquivos foram anexados corretamente. Siga com a validação dos dados apresentados abaixo."
        color = 'green'
    elif msg ==1:
        textoUpload = "Favor selecionar todos os arquivos antes de seguir para os próximos passos."
        color = 'red'
    Label(canvas, text=textoUpload, foreground=color).grid(row=4, columnspan=3, pady=10)
    infosContrato(contrato)
    infosSimulador(simulador)
    infosLaudo(laudo)



######## Botões relacionados ao Contrato ########
textBoxContrato = Entry(
    canvas, 
    width=30
    )
textBoxContrato.grid(row=0, column=2)    
botaoContrato = Label(
    canvas, 
    text='Selecione o contrato em PDF'
    )
botaoContrato.grid(row=0, column=0, padx=10)
anexarContrato = Button(
    canvas, 
    text ='Selecione o Contrato', 
    command = lambda:open_file_pdf(textBoxContrato)
    ) 
anexarContrato.grid(row=0, column=1)

######## Botões Simulador ########
textBoxSimulador = Entry(
    canvas,
    width=30,
    
    )
textBoxSimulador.grid(row=1, column=2)    
botaoSimulador = Label(
    canvas, 
    text='Selecione o simulador em Excel'
    )
botaoSimulador.grid(row=1, column=0, padx=10)
anexarSimulador = Button(
    canvas, 
    text ='Selecione o Simulador', 
    command = lambda:open_file_excel()
    ) 
anexarSimulador.grid(row=1, column=1)

######## Botões Laudo ########
textBoxLaudo = Entry(
    canvas,
    width=30
    )
textBoxLaudo.grid(row=2, column=2)
botaoLaudo = Label(
    canvas, 
    text='Selecione o Laudo em PDF'
    )
botaoLaudo.grid(row=2, column=0, padx=10)
anexarLaudo = Button(
    canvas, 
    text ='Selecione o Laudo', 
    command = lambda:open_file_pdf(textBoxLaudo)
    ) 
anexarLaudo.grid(row=2, column=1)

######## Botão Importar Valores ########
upld = tk.Button(
    canvas, 
    text='Importar Valores', 
    command=uploadFiles
    )
upld.grid(row=3, columnspan=1, pady=10)

######### Botão Validar ########
conf = Button(
    canvas, 
    text='Validar Valores', 
    command=validarEntry
    )
conf.grid(row=3, columnspan=5, pady=10)

######## Menu Dados da Operação ########
dadosOperacao = Label(
    canvas, 
    text='Dados da Operação',
    font=("bold",15),
    )
dadosOperacao.grid(row=6, column=0, pady=10)

######## Menu Superior Dados da Operação ########
dadosContrato = Label(
    canvas, 
    text='Contrato',
    font=40
    )
dadosContrato.grid(row=7, column=0, pady=10)
dadosSimulador = Label(
    canvas, 
    text='Simulador',
    font=40,
    )
dadosSimulador.grid(row=7, column=4, padx=30)


######## Menu Inferior Dados do Imóvel ########
dadosImovel = Label(
    canvas, 
    text='Dados do imóvel',
    font=("bold",15),
    )
dadosImovel.grid(row=23, column=0, padx=30, pady=30)
dadosLaudo = Label(
    canvas, 
    text='Dados do Laudo',
    font=40,
    )
dadosLaudo.grid(row=23, column=7, padx=30)

######## Menu Participantes da Operação ########
dadosParticipantesOps = Label(
    canvas, 
    text='Participantes da Operação',
    font=("bold",15),
    )
dadosParticipantesOps.grid(row=28, column=0, padx=30, pady=30)


######## Saída de Dados Contratos ########
try:
    #Valor Bruto
    try:
        botaovalorBrutoC = Label(
            canvas, 
            text='Valor Bruto'
            )
        botaovalorBrutoC.grid(row=9, column=0)
        tbCValorBruto = Entry(
            canvas,
            width=20
            )
        tbCValorBruto.grid(row=9, column=1)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoC = Label(
            canvas, 
            text='Valor Liquido'
            )
        botaovalorLiquidoC.grid(row=10, column=0)
        tbCValorLiquido = Entry(
            canvas,
            width=20
            )
        tbCValorLiquido.grid(row=10, column=1)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoC = Label(
            canvas, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoC.grid(row=11, column=0)
        tbCAmortizacao = Entry(
            canvas,
            width=20
            )
        tbCAmortizacao.grid(row=11, column=1)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceC = Label(
            canvas, 
            text='Indice de Correção'
            )
        botaoIndiceC.grid(row=12, column=0)
        tbCIndice = Entry(
            canvas,
            width=20
            )
        tbCIndice.grid(row=12, column=1)
    except:
        pass

    #Prazo
    try:
        botaoPrazoC = Label(
            canvas, 
            text='Prazo da Operação'
            )
        botaoPrazoC.grid(row=13, column=0)
        tbCPrazo = Entry(
            canvas,
            width=20
            )
        tbCPrazo.grid(row=13, column=1)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasC = Label(
            canvas, 
            text='Número de parcelas'
            )
        botaoParcelasC.grid(row=14, column=0)
        tbCParcelas = Entry(
            canvas,
            width=20
            )
        tbCParcelas.grid(row=14, column=1)
    except:
        pass

    #Data Desembolso
    try:
        botaoDataDesemC = Label(
            canvas, 
            text='Data de Desembolso'
            )
        botaoDataDesemC.grid(row=15, column=0)
        tbCDataDesem = Entry(
            canvas,
            width=20
            )
        tbCDataDesem.grid(row=15, column=1)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaC = Label(
            canvas, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaC.grid(row=16, column=0)
        tbCDataPriParcela = Entry(
            canvas,
            width=20
            )
        tbCDataPriParcela.grid(row=16, column=1)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaC = Label(
            canvas, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaC.grid(row=17, column=0)
        tbCDataUltimaParcela = Entry(
            canvas,
            width=20
            )
        tbCDataUltimaParcela.grid(row=17, column=1)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParC = Label(
            canvas, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParC.grid(row=18, column=0)
        tbCValorPriPar = Entry(
            canvas,
            width=20
            )
        tbCValorPriPar.grid(row=18, column=1)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosC = Label(
            canvas, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosC.grid(row=19, column=0)
        tbCValorPriEncargosPar = Entry(
            canvas,
            width=20
            )
        tbCValorPriEncargosPar.grid(row=19, column=1)
    except:
        pass

    #Data Contrato
    try:
        botaoDataContratoC = Label(
            canvas, 
            text='Data do Contrato'
            )
        botaoDataContratoC.grid(row=20, column=0)
        tbCDataContrato = Entry(
            canvas,
            width=20
            )
        tbCDataContrato.grid(row=20, column=1)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoC = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoC.grid(row=24, column=0)
        tbCValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbCValorAvaliacao.grid(row=24, column=1)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaC = Label(
            canvas, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaC.grid(row=25, column=0)
        tbCMatricula = Entry(
            canvas,
            width=20
            )
        tbCMatricula.grid(row=25, column=1)
    except:
        pass

    #Cartório
    try:
        botaoCartorioC = Label(
            canvas, 
            text='Cartório'
            )
        botaoCartorioC.grid(row=26, column=0)
        tbCCartorio = Entry(
            canvas,
            width=20
            )
        tbCCartorio.grid(row=26, column=1)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")


######## Saída de Dados Simulador ########
try:
    #Valor Bruto
    try:
        botaovalorBrutoS = Label(
            canvas, 
            text='Valor Bruto'
            )
        botaovalorBrutoS.grid(row=9, column=4)
        tbSValorBruto = Entry(
            canvas,
            width=20
            )
        tbSValorBruto.grid(row=9, column=5)
    except:
        pass

    #Valor Liquido
    try:
        botaovalorLiquidoS = Label(
            canvas, 
            text='Valor Liquido'
            )
        botaovalorLiquidoS.grid(row=10, column=4)
        tbSValorLiquido = Entry(
            canvas,
            width=20
            )
        tbSValorLiquido.grid(row=10, column=5)
    except:
        pass

    #Sistena de Amortização
    try:
        botaoSistemaAmortizacaoS = Label(
            canvas, 
            text='Sistema de Amortização'
            )
        botaoSistemaAmortizacaoS.grid(row=11, column=4)
        tbSAmortizacao = Entry(
            canvas,
            width=20
            )
        tbSAmortizacao.grid(row=11, column=5)
    except:
        pass

    #Índice de Correção
    try:
        botaoIndiceS = Label(
            canvas, 
            text='Indice de Correção'
            )
        botaoIndiceS.grid(row=12, column=4)
        tbSIndice = Entry(
            canvas,
            width=20
            )
        tbSIndice.grid(row=12, column=5)
    except:
        pass

    #Prazo
    try:
        botaoPrazoS = Label(
            canvas, 
            text='Prazo da Operação'
            )
        botaoPrazoS.grid(row=13, column=4)
        tbSPrazo = Entry(
            canvas,
            width=20
            )
        tbSPrazo.grid(row=13, column=5)
    except:
        pass

    #Número de Parcelas
    try:
        botaoParcelasS = Label(
            canvas, 
            text='Número de parcelas'
            )
        botaoParcelasS.grid(row=14, column=4)
        tbSParcelas = Entry(
            canvas,
            width=20
            )
        tbSParcelas.grid(row=14, column=5)
    except:
        pass

    #Data Desembolso
    try:
        botaoDataDesemS = Label(
            canvas, 
            text='Data de Desembolso'
            )
        botaoDataDesemS.grid(row=15, column=4)
        tbSDataDesem = Entry(
            canvas,
            width=20
            )
        tbSDataDesem.grid(row=15, column=5)
    except:
        pass

    #Data Primeira Parcela
    try:
        botaoDataPriParcelaS = Label(
            canvas, 
            text='Data Primeira Parcela'
            )
        botaoDataPriParcelaS.grid(row=16, column=4)
        tbSDataPriParcela = Entry(
            canvas,
            width=20
            )
        tbSDataPriParcela.grid(row=16, column=5)
    except:
        pass

    #Data Última Parcela
    try:
        botaoDataUltimaParcelaS = Label(
            canvas, 
            text='Data Última Parcela'
            )
        botaoDataUltimaParcelaS.grid(row=17, column=4)
        tbSDataUltimaParcela = Entry(
            canvas,
            width=20
            )
        tbSDataUltimaParcela.grid(row=17, column=5)
    except:
        pass

    #Data Valor Primeira Parcela
    try:
        botaoValorPriParS = Label(
            canvas, 
            text='Valor Primeira Parcela (Juros + Amortização)'
            )
        botaoValorPriParS.grid(row=18, column=4)
        tbSValorPriPar = Entry(
            canvas,
            width=20
            )
        tbSValorPriPar.grid(row=18, column=5)
    except:
        pass

    #Data Valor Primeira Parcela + Encargos
    try:
        botaoValorPriParEncargosS = Label(
            canvas, 
            text='Valor Primeira Parcela'
            )
        botaoValorPriParEncargosS.grid(row=19, column=4)
        tbSValorPriEncargosPar = Entry(
            canvas,
            width=20
            )
        tbSValorPriEncargosPar.grid(row=19, column=5)
    except:
        pass

    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoS = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoS.grid(row=24, column=4)
        tbSValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbSValorAvaliacao.grid(row=24, column=5)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

######## Saída de Dados Laudo ########
try:
    #Valor de Avaliação do Imóvel
    try:
        botaoValorAvaliacaoL = Label(
            canvas, 
            text='Valor de Avaliação do Imóvel'
            )
        botaoValorAvaliacaoL.grid(row=24, column=7, padx=40)
        tbLValorAvaliacao = Entry(
            canvas,
            width=20
            )
        tbLValorAvaliacao.grid(row=24, column=8)
    except:
        pass

    #Número da Matrícula
    try:
        botaoMatriculaL = Label(
            canvas, 
            text='Matrícula do Imóvel'
            )
        botaoMatriculaL.grid(row=25, column=7)
        tbLMatricula = Entry(
            canvas,
            width=20
            )
        tbLMatricula.grid(row=25, column=8)
    except:
        pass

    #Cartório
    try:
        botaoCartorioL = Label(
            canvas, 
            text='Cartório'
            )
        botaoCartorioL.grid(row=26, column=7)
        tbLCartorio = Entry(
            canvas,
            width=20
            )
        tbLCartorio.grid(row=26, column=8)
    except:
        pass
except:
    print("Erro ao gerar menu contrato")

######## Saída de dados dos participantes ########

#Número de Participantes
try:
    botaoNumeroParticipantes = Label(
        canvas, 
        text='Número de Participantes'
        )
    botaoNumeroParticipantes.grid(row=29, column=0, padx=20,pady=5)
    tbCNumParticipantes = Entry(
        canvas,
        width=20
        )
    tbCNumParticipantes.grid(row=29, column=1)
    botaoPuxarParticipantes = Button(
    canvas, 
    text ='Importar Participantes', 
    command = lambda:camposParcipantes()
    ) 
    botaoPuxarParticipantes.grid(row=29, column=2)
except:
    pass


#Teste

def getEntry():
    a = 20
    teste = list(canvas.children.values())[a]
    print(teste)
    print()

### Demais Informações ###
def camposParcipantes():
    try:
        
        listaParticipantes = {"nomeCompletoP1":"","cpfP1":"","dataNascimentoP1":"","cidadeP1":"","endereçoP1":"","numeroResidenciaP1":"","complementoP1":"",
                            "bairroP1":"","estadoP1":"","cepP1":"","telefoneP1":"","emailP1":"","tipoOperacaoP1":"","sexoP1":"","relacaoDoP1":"",
                            "participacaoNaOperacaoP1":"","nomeCompletoP2":"","cpfP2":"","dataNascimentoP2":"","cidadeP2":"","endereçoP2":"",
                            "numeroResidenciaP2":"","complementoP2":"","bairroP2":"","estadoP2":"","cepP2":"","telefoneP2":"","emailP2":"",
                            "tipoOperacaoP2":"","sexoP2":"","relacaoDoP2":"","participacaoNaOperacaoP2":"","nomeCompletoP3":"","cpfP3":"",
                            "dataNascimentoP3":"","cidadeP3":"","endereçoP3":"","numeroResidenciaP3":"","complementoP3":"","bairroP3":"",
                            "estadoP3":"","cepP3":"","telefoneP3":"","emailP3":"","tipoOperacaoP3":"","sexoP3":"","relacaoDoP3":"",
                            "participacaoNaOperacaoP3":"","nomeCompletoP4":"","cpfP4":"","dataNascimentoP4":"","cidadeP4":"","endereçoP4":"",
                            "numeroResidenciaP4":"","complementoP4":"","bairroP4":"","estadoP4":"","cepP4":"","telefoneP4":"","emailP4":"",
                            "tipoOperacaoP4":"","sexoP4":"","relacaoDoP4":"","participacaoNaOperacaoP4":"","nomeCompletoP5":"","cpfP5":"","dataNascimentoP5":"",
                            "cidadeP5":"","endereçoP5":"","numeroResidenciaP5":"","complementoP5":"","bairroP5":"","estadoP5":"","cepP5":"","telefoneP5":"",
                            "emailP5":"","tipoOperacaoP5":"","sexoP5":"","relacaoDoP5":"","participacaoNaOperacaoP5":"","nomeCompletoP6":"","cpfP6":"",
                            "dataNascimentoP6":"","cidadeP6":"","endereçoP6":"","numeroResidenciaP6":"","complementoP6":"","bairroP6":"","estadoP6":"","cepP6":"",
                            "telefoneP6":"","emailP6":"","tipoOperacaoP6":"","sexoP6":"","relacaoDoP6":"","participacaoNaOperacaoP6":"","nomeCompletoP7":""
                            ,"cpfP7":"","dataNascimentoP7":"","cidadeP7":"","endereçoP7":"","numeroResidenciaP7":"","complementoP7":"","bairroP7":"","estadoP7":"",
                            "cepP7":"","telefoneP7":"","emailP7":"","tipoOperacaoP7":"","sexoP7":"","relacaoDoP7":"","participacaoNaOperacaoP7":"",
                            "nomeCompletoP8":"","cpfP8":"","dataNascimentoP8":"","cidadeP8":"","endereçoP8":"","numeroResidenciaP8":"","complementoP8":"",
                            "bairroP8":"","estadoP8":"","cepP8":"","telefoneP8":"","emailP8":"","tipoOperacaoP8":"","sexoP8":"","relacaoDoP8":"",
                            "participacaoNaOperacaoP8":"","nomeCompletoP9":"","cpfP9":"","dataNascimentoP9":"","cidadeP9":"","endereçoP9":"","numeroResidenciaP9":"",
                            "complementoP9":"","bairroP9":"","estadoP9":"","cepP9":"","telefoneP9":"","emailP9":"","tipoOperacaoP9":"","sexoP9":"","relacaoDoP9":"",
                            "participacaoNaOperacaoP9":"","nomeCompletoP10":"","cpfP10":"","dataNascimentoP10":"","cidadeP10":"","endereçoP10":"","numeroResidenciaP10":"",
                            "complementoP10":"","bairroP10":"","estadoP10":"","cepP10":"","telefoneP10":"","emailP10":"","tipoOperacaoP10":"","sexoP10":"",
                            "relacaoDoP10":"","participacaoNaOperacaoP10":""
                        }																			
        
        #Participante 1
        nParticipante = int(tbCNumParticipantes.get())
        
        i=1
        j=0

        while i <= nParticipante:
            print(i)
            
            if i>1:
                j=5+j
            

            #Definir Número de Participantes
            participante1 = Label(
                canvas, 
                text=f'Participante {i}',
                font=("bold",15),
                )
            participante1.grid(row=30+j, column=0,pady=20)

                #Nome Participante
            try:
                botaoNomeP1C = Label(
                    canvas, 
                    text='Nome Completo'
                    )
                botaoNomeP1C.grid(row=31+j, column=0, padx=20)
                tbCNomeP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCNomeP1.grid(row=31+j, column=1, pady=0)
            except:
                pass

            #CPF Participante 1
            try:
                botaoCpfP1C = Label(
                    canvas, 
                    text='CPF'
                    )
                botaoCpfP1C.grid(row=31+j, column=2, padx=20)
                tbCCpfP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCCpfP1.grid(row=31+j, column=3, pady=0)
            except:
                pass

            #Data Nascimento Participante 1
            try:
                botaoDataNascimentoP1C = Label(
                    canvas, 
                    text='Data Nascimento'
                    )
                botaoDataNascimentoP1C.grid(row=31+j, column=4, padx=20)
                tbCDataNascimentoP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCDataNascimentoP1.grid(row=31+j, column=5, pady=0)
            except:
                pass

            #Cidade Participante 1
            try:
                botaoCidadeP1C = Label(
                    canvas, 
                    text='Cidade'
                    )
                botaoCidadeP1C.grid(row=31+j, column=6, padx=20)
                tbCCidadeP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCCidadeP1.grid(row=31+j, column=7, pady=0)
            except:
                pass

            #Endereço Participante 1
            try:
                botaoEnderecoP1C = Label(
                    canvas, 
                    text='Endereço'
                    )
                botaoEnderecoP1C.grid(row=31+j, column=8, padx=20)
                tbCEnderecodP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCEnderecodP1.grid(row=31+j, column=9, pady=0)
            except:
                pass

            #Número Participante 1
            try:
                botaoNumeroP1C = Label(
                    canvas, 
                    text='Numero'
                    )
                botaoNumeroP1C.grid(row=32+j, column=0, padx=20)
                tbCNumeroP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCNumeroP1.grid(row=32+j, column=1, pady=5)
            except:
                pass

            #Complemento Participante 1
            try:
                botaoComplementoP1C = Label(
                    canvas, 
                    text='Complemento'
                    )
                botaoComplementoP1C.grid(row=32+j, column=2, padx=20)
                tbCComplementoP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCComplementoP1.grid(row=32+j, column=3, pady=0)
            except:
                pass

            #Bairro Participante 1
            try:
                botaoBairroP1C = Label(
                    canvas, 
                    text='Bairro'
                    )
                botaoBairroP1C.grid(row=32+j, column=4, padx=20)
                tbCBairroP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCBairroP1.grid(row=32+j, column=5, pady=0)
            except:
                pass

            #UF Participante 1
            try:
                botaoUFP1C = Label(
                    canvas, 
                    text='UF     '
                    )
                botaoUFP1C.grid(row=32+j, column=6, padx=20)
                tbCUFP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCUFP1.grid(row=32+j, column=7, pady=0)
            except:
                pass

            #CEP Participante 1
            try:
                botaoCEPP1C = Label(
                    canvas, 
                    text='CEP'
                    )
                botaoCEPP1C.grid(row=32+j, column=8, padx=20)
                tbCCEPP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCCEPP1.grid(row=32+j, column=9, pady=0)
            except:
                pass

            #Telefone Participante 1
            try:
                botaoTelefoneP1C = Label(
                    canvas, 
                    text='Telefone'
                    )
                botaoTelefoneP1C.grid(row=33+j, column=0, padx=20)
                tbCTelefoneP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCTelefoneP1.grid(row=33+j, column=1, pady=0)
            except:
                pass

            #Email Participante 1
            try:
                botaoEmailP1C = Label(
                    canvas, 
                    text='Email'
                    )
                botaoEmailP1C.grid(row=33+j, column=2, padx=20)
                tbCEmailP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCEmailP1.grid(row=33+j, column=3, pady=0)
            except:
                pass

            #Tipo de Operação
            try:
                tipoOpP1C = Label(
                    canvas, 
                    text='Tipo Operação',
                    )
                tipoOpP1C.grid(row=33+j, column=4)
                variable = StringVar(canvas)
                w = OptionMenu(canvas, variable, "", "PF", "PJ")
                w.grid(row=33+j, column=5)
            except:
                pass

            #Sexo Operação
            try:
                sexoP1C = Label(
                    canvas, 
                    text='Sexo'
                    )
                sexoP1C.grid(row=33+j, column=6)
                variableSexo = StringVar(canvas)
                canvasexo = OptionMenu(canvas, variableSexo, "", "F", "M")
                canvasexo.grid(row=33+j, column=7)
            except:
                pass

            #Relação
            try:
                relacaoP1C = Label(
                    canvas, 
                    text='Relação'
                    )
                relacaoP1C.grid(row=33+j, column=8)
                variableRelacao = StringVar(canvas)
                wRelacao = OptionMenu(canvas, variableRelacao, "", "Titular", "Cônjuge", "Procurador" ,"Pai", "Mãe", "Irmão")
                wRelacao.grid(row=33+j, column=9)
            except:
                pass

            #Participação
            try:
                participacaoP1C = Label(
                    canvas, 
                    text='Participação'
                    )
                participacaoP1C.grid(row=34+j, column=0, padx=20)
                tbCparticipacaoP1 = Entry(
                    canvas,
                    width=20
                    )
                tbCparticipacaoP1.grid(row=34+j, column=1, pady=5)
            except:
                pass

            #### Adicionar Mais um ####
            i+= 1 
        
        #Passo 3 - Adicionar a barra de rolagem ao canvas
        scrollbar = Scrollbar(main_frame, orient=VERTICAL, command=my_canvas.yview)
        scrollbar.pack(side=RIGHT, fill=Y)

        #Add Button
        botaoEnviarDados = Button(
            canvas,
            text ='Enviar para Aztronic',
            command = lambda:getEntry()
        )
        botaoEnviarDados.grid(row=34+j+1, column=0, pady=35)


        #Passo 4 - Configurar o canvas
        my_canvas.configure(yscrollcommand=scrollbar.set)
        my_canvas.bind('<Configure>', lambda e:my_canvas.configure(scrollregion=my_canvas.bbox("all")))


    except:
        
        pass


#Rodar Programa
canvas.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Matheus\AppData\Local\Temp\ipykernel_23132\2605528895.py", line 340, in uploadFiles
    infosContrato(contrato)
  File "C:\Users\Matheus\AppData\Local\Temp\ipykernel_23132\2605528895.py", line 97, in infosContrato
    valorBrutoC = str(textoContrato['valorTotal']).replace(".",",")
KeyError: 'valorTotal'
